In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from scipy.stats import pearsonr

In [ ]:
# COMPUTE MOTION ENERGY FROM DLC BODY VECTORS
path = r'/Volumes/Behaviour 3/Bodyvectors_05_2000'

bodies = [body for body in os.listdir(path) if body.endswith('bodyvectors_07_2000.csv')] # output from DLC analysis code

# using the header from a trial where all 612 pre- and 632 post-stimulus frames passed the 0.5 likelihood to get
# a list ranging from -612 to 632
#correct = pd.read_csv(r'I:\Data to test Aras 2P code\Bodyvectors_07_2000\2P05_Ses01_25OCT2021_approach_Trial_1_89009_bodyvectors_07_2000.csv')
correct = pd.read_csv('/Volumes/Behaviour 3/Bodyvectors_05_2000/2P05_Ses01_25OCT2021_approach_Trial_1_89009_bodyvectors_07_2000.csv')
cor_cols = list(correct.columns)

b_motion = pd.DataFrame()
b_motion2 = pd.DataFrame()

bigcount = 0
short_vectors = []
shorter_vectors = []
not_19 = []

#the number of camera frames acquired during each 2P frame
bins = [65,66,65,66,66,65,66,65,66,65,66,66,65,66,65,66,65,66,65]

for body in bodies:
    print('procesing: ', body)
    if body.startswith('._'): # circumventing issues caused by invisible files on the external drive
        continue
      
    b = pd.read_csv(os.path.join(path,body))
    # insert NaN columns for the frames with likelihood <0.5 in order to have vectors of identical shape
    if b.shape[1] < 1245:
        length = b.shape[1]
        print(length, 'bodyvector is short')
        short_vectors.append([body, length])
        smallcount = 0
        cols = list(b.columns)
        for c, cor_col in enumerate(cor_cols):
            if cor_col in cols:
                continue
            else: 
                if int(cor_col) > 0:
                    smallcount += 1 # counting the number of missing post-stimulus frames 
                b[cor_col]=np.nan
    
        print('smallcount: ', smallcount)
        
        if smallcount > 63: # skip trials that have more than 10 % of post-stimulus frames missing
            print(body, ' doesnt have enough motion frames')
            bigcount += 1
            shorter_vectors.append([body, 1245-length, smallcount])
            continue
        b = b[cor_cols] 
        
    b_mo = b.diff(axis=1).abs() # compute frame-wise motion energy
    b_mo = b_mo.mean(axis=0) # calculate mean motion energy across body parts
    b_mo[0] = np.nan # insert NaN column to maintain vector shape
    print(b_mo.shape)
    
    a = 0
    downsampled = []
    for b in bins:
        downsampled.append(b_mo.iloc[a:a+b].mean())
        a = a + b
        
    if np.isnan(downsampled).any(): #not all trials had a sample for each bin. we might need to reduce the likelihood now?
        print('Cannot use: there are not 19 bins')
        not_19.append(body)
        continue
    else:
        b_motion['motion_e'] = downsampled
        entries = body.split('_')
        identifier = entries[0]+'_'+entries[1]+'_'+entries[2]+'_'+entries[3]+'_'+entries[4]+'_'+entries[5]+'_'+entries[6]
        b_motion['identifier'] = identifier
        b_motion2 = pd.concat([b_motion2, b_motion], axis=0)

b_motion2['frame'] = b_motion2.index
print('number of short bodyvectors :', bigcount)
short_bodyvectors = pd.DataFrame(short_vectors, columns=('trial', 'nb of frames')) # list of trials with any missing frame
shorter_bodyvectors = pd.DataFrame(shorter_vectors, columns=('trial', 'total missing frames', 'post-stim missing frames')) # list of trials that were skipped because of >10% missing frames
not_19 = pd.DataFrame(not_19)

b_motion2.to_csv(os.path.join(path, 'body_motion_energy_05_2000.csv'))
short_bodyvectors.to_csv(os.path.join(path, 'short_bodyvectors_05_2000.csv'))
shorter_bodyvectors.to_csv(os.path.join(path, 'dropped_bodyvectors_05_2000.csv'))
not_19.to_csv(os.path.join(path, 'not_19_05_2000.csv'))

In [ ]:
# MATCH BODY MOTION ENERGY TO NEURAL ACTIVITY FOR EACH CELL AND FRAME SESSION-WISE
motion = pd.read_csv(r'/Volumes/Behaviour 3/Bodyvectors_05_2000/body_motion_energy_05_2000.csv', index_col=0)

input_path = r'/Volumes/Behaviour/Tailored 3sec 2P Trials'# dF_F0 traces cropped to 1.5 sec pre- and post-stimulus

sessions = [session for session in os.listdir(input_path) if session.endswith('.csv')]

pearsons = pd.DataFrame()
session_names = []

for session in sessions:
    if session.startswith('._'): # circumventing issues with invisible files on external drives
        continue
    if session.endswith('2P05_Ses01.csv'): # excluding for the time being incomplete sessions (half-sessions)
        continue
    if session.endswith('2P05_Ses02.csv'):
        continue
    if session.endswith('2P07_Ses02.csv'):
        continue
    if session.endswith('2P11_Ses03.csv'):
        continue
    #if session.endswith('CFA.csv'): # excluding for the time being CFA sessions
        #continue
    
    print('processing: ', session)
    entries = session.split('_')
    
    if session.endswith('CFA.csv'): 
        ses_name = entries[3]+'_'+entries[4][:3]
        
    else:
        ses_name = entries[3]+'_'+entries[4][:5]
            
    print(ses_name)
    session_names.append(ses_name)
    cells = pd.read_csv(os.path.join(input_path,session), index_col=0)
    print('cells shape', cells.shape)
    cell_list = (list(cells.columns)) # list of rois 
    cell_list = cell_list[:-1] # last column is the identifier, hence I remove it
    session_mo = motion[motion['identifier'].str.contains(str(ses_name))] # subsetting body motion energy file to the session being processed
    print('session_mo shape', session_mo.shape)# subsetting body motion energy file to the session being processed
    session_mo['ids']  = session_mo.apply(lambda x: x['identifier'] +'_'+str(x['frame']), axis=1) # creating a new identifier column which combines trial ID and frame number
    # now I have to create a matching frames_for_each_trial column for the 2P recording dataframe
    frames_template = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]) # template with the number of frames per trial
    factor = int(cells.shape[0]/19) # finding out how many trials per session there are (this is not always identical to the motion energy file, due to the likelyhood threshold)
    cells['frame'] = np.tile(frames_template, factor) # distritbute 'frames_template' across the whole 2P recording
    cells['ids'] = cells.apply(lambda x: x['identifier'] +'_'+str(x['frame']), axis=1) # create new identifier column as in line 34
    temp = pd.merge(cells, session_mo, on="ids") # creating a temporary data frame by merging motion energy vectors and 2P traces aligned on 'ids' column  
    temp = temp.dropna()   # then dropping those frames that don't have motion energy values due to likelihood threshold       
    
    li = []
    for c in cell_list: # for each cell in a session, compute Pearsons' r by correlating with motion energy
        P,_ = pearsonr(temp[c],temp['motion_e'])
        li.append((c, P))
    
    corr = pd.DataFrame(li, columns=('cell_id', 'pearsons'))
    
    pearsons = pd.concat([pearsons, corr], axis = 0)

pearsons.to_csv(os.path.join(path, 'pearsons_long_05_2000.csv'))

In [ ]:
# To make correlation plots off the long-format data-frame
output_path = '/Volumes/Behaviour/Tailored 3sec 2P Trials/Motion Correlation Plots'

for s in session_names:
    print(s)
    df = pearsons[pearsons['cell_id'].str.contains(str(s))]
    plt.figure(figsize=(10,5))
    plt.scatter(x=df['cell_id'], y=df['pearsons'], alpha = 0.5)
    plt.ylim((-0.2,0.5))
    plt.show()
    #plt.savefig(os.path.join(output_path, s+'_motion_correlation_05.png'))